## Load data

In [1]:
import pandas
urlTraining = 'https://raw.githubusercontent.com/gbrian/machinelearning_exercises/master/ml/supervised/exercise_1/test.tsv'
urlTest = 'https://raw.githubusercontent.com/gbrian/machinelearning_exercises/master/ml/supervised/exercise_1/test.tsv'
orgtraining = pandas.read_csv(urlTraining, delimiter='\t')
orgtesting = pandas.read_csv(urlTest, delimiter='\t')
orgtraining.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,Class
0,1,1,2,40,10,101,1,5,2,1,6,2,5
1,1,1,2,43,10,7,10,1,44,10,2,44,5
2,1,1,43,40,10,35,11,4,45,11,4,45,20
3,1,1,43,41,10,13,10,7,42,10,7,42,5
4,1,1,43,43,10,44,12,6,49,12,6,49,20


## Feature selection
https://machinelearningmastery.com/feature-selection-machine-learning-python/


In [4]:
testing = orgtesting.drop('A1', axis=1)
training = orgtraining.drop('A1', axis=1)

In [27]:
# Feature Extraction with Univariate Statistical Tests (Chi-squared for classification)
import pandas
import numpy
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
classIndex = len(training.columns)-1
lastColIndex = classIndex-1
array = training.values
X = array[:,0:-2]
Y = array[:,-1]
# feature extraction
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, Y)
# summarize scores
columns = list(training.columns)
numpy.set_printoptions(precision=3)
scoredFeatures = [{"score":x,"column":columns[i]} for i,x in enumerate(fit.scores_)]
sortedScoredFeatures = sorted(scoredFeatures, key=lambda x: x['score'], reverse=True) 
print(pandas.DataFrame.from_dict(sortedScoredFeatures))


   column         score
0      A1  6.922209e+07
1      A2  1.176584e+07
2      A3  9.947493e+06
3      A4  3.012508e+05
4      A6  1.659314e+05
5      A9  7.330293e+04
6      A5  6.170595e+04
7      A7  1.552336e+04
8     A10  1.549642e+04
9      A8  2.239109e+02
10    A11  1.973365e+02


## Classification
### auto_ml
http://auto-ml.readthedocs.io/en/latest

In [23]:
# http://auto-ml.readthedocs.io/en/latest/
from auto_ml import Predictor

# If you pass in any categorical data as a number, tell us here and we'll take care of it.
col_desc_dictionary = {"Class": "output"}

# Can pass in type_of_estimator='regressor' as well
ml_predictor = Predictor(type_of_estimator='classifier', column_descriptions=col_desc_dictionary)

# Wait for the machine to learn all the complex and beautiful patterns in your data...
ml_predictor.train(training, ml_for_analytics=True)

# And this time, in your shell, it will print out the results for what it found was useful in making predictions!

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at https://github.com/ClimbsRocks/auto_ml
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}
Running basic data cleaning
Performing feature scaling
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}


********************************************************************************************
About to fit the pipeline for the model GradientBoostingClassifier to predict Cla

[229] random_holdout_set_from_training_data's score is: 0.698
[232] random_holdout_set_from_training_data's score is: 0.698
[235] random_holdout_set_from_training_data's score is: 0.699
[238] random_holdout_set_from_training_data's score is: 0.698
[241] random_holdout_set_from_training_data's score is: 0.699
[244] random_holdout_set_from_training_data's score is: 0.699
[247] random_holdout_set_from_training_data's score is: 0.699
[250] random_holdout_set_from_training_data's score is: 0.699
[255] random_holdout_set_from_training_data's score is: 0.7
[260] random_holdout_set_from_training_data's score is: 0.701
[265] random_holdout_set_from_training_data's score is: 0.701
[270] random_holdout_set_from_training_data's score is: 0.702
[275] random_holdout_set_from_training_data's score is: 0.704
[280] random_holdout_set_from_training_data's score is: 0.703
[285] random_holdout_set_from_training_data's score is: 0.702
[290] random_holdout_set_from_training_data's score is: 0.702
[295] rand

C:\Users\gustavo.brian\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


The printed list will only contain at most the top 100 features.
The full analytics results will be saved to a filed called: auto_ml_analytics_results_Class.csv

+--------+----------------+--------------+----------+-------------------+-------------------+-----------+-----------+-----------+-----------+
|        | Feature Name   |   Importance |    Delta |   FR_Decrementing |   FR_Incrementing |   FRD_abs |   FRI_abs |   FRD_MAP |   FRI_MAP |
|--------+----------------+--------------+----------+-------------------+-------------------+-----------+-----------+-----------+-----------|
| 9.0000 | A7             |       0.0081 |   2.0897 |            0.0023 |           -0.0010 |    0.0049 |    0.0031 |    0.0003 |    0.0002 |
| 8.0000 | A10            |       0.0100 |   2.0916 |           -0.0023 |           -0.0001 |    0.0049 |    0.0040 |    0.0003 |    0.0003 |
| 7.0000 | A11            |       0.0344 |   1.0142 |            0.0020 |           -0.0025 |    0.0064 |    0.0079 |    0.0011 

In [21]:
# Where new_data is a single dictionary, or a DataFrame
predictions = ml_predictor.predict(testing)
classes = [[i,k] for i,k in enumerate(testing['Class'])]
validation = list(map(lambda p: [p[1],predictions[p[0]],p[1] == predictions[p[0]]], classes))
print(validation[0:5])
total = len(validation)
ok = len(list(filter(lambda k: k[2], validation)))
perc = 100 / total * ok
print("Validation %d of %d - %f%%" % (ok, total, perc))

[[5, 0.9999998807907104, False], [5, 0.9999998807907104, False], [20, 0.9999998807907104, False], [5, 0.9999998807907104, False], [20, 0.9999998807907104, False]]
Validation 0 of 25092 - 0.000000%


### TPOT
https://rhiever.github.io/tpot


In [5]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

X_train = training[training.columns[:-1]]
X_test = testing[testing.columns[:-1]]
y_train = training[training.columns[-1]]
y_test = testing[testing.columns[-1]]

pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2)
pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))
pipeline_optimizer.export('tpot_exported_pipeline.py')

ModuleNotFoundError: No module named 'tpot'

## Build classifier

In [83]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split

features = training.drop('Class', axis=1).values
training_target = training['Class'].values

# Score on the training set was:0.8861336642426755
exported_pipeline = ExtraTreesClassifier(bootstrap=False, 
                                            criterion="entropy", max_features=0.8500000000000001, min_samples_leaf=20, min_samples_split=15, n_estimators=100)

exported_pipeline.fit(features, training_target)

t = testing.drop('Class', axis=1)
try:
    testing.drop('predict', axis=1)
except:
    pass
t = t.values
testing['predict'] = exported_pipeline.predict(t)
testing.to_csv('prediction.csv')
print(testing.head())
ok= len(testing[testing.Class == testing.predict].values)
total = len(t)
accuracy = 100 / total * ok
print("Ok %d / %d %f%%" % (ok, total, accuracy))


   A1  A2  A3  A4  A5   A6  A7  A8  A9  A10  A11  A12  Class  predict
0   1   1   2  40  10  101   1   5   2    1    6    2      5        5
1   1   1   2  43  10    7  10   1  44   10    2   44      5        5
2   1   1  43  40  10   35  11   4  45   11    4   45     20        5
3   1   1  43  41  10   13  10   7  42   10    7   42      5        5
4   1   1  43  43  10   44  12   6  49   12    6   49     20        5
Ok 22237 / 25092 88.621872%


### Test predictor service
run `python PredictorService.py`

In [89]:
import requests 

requests.get('http://localhost:5002/test').text

'"Ok!"\n'

In [3]:
import sys
print(sys.version)

3.5.2 |Anaconda custom (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]
